In [1]:
## This code was inspired from the orginal Pix2Pix implementation by tensorflow (https://www.tensorflow.org/tutorials/generative/pix2pix) and CycleGAn implementation (#https://keras.io/examples/generative/cyclegan/)
#Imports
!pip install tensorflow-addons
!pip install visualkeras
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow.python import training
import visualkeras

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_addons as tfa

import os
import time
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 23.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 992 kB 29.6 MB/s 


In [2]:
problem = "MMB" #@param ["MMB", "Cantilever"]

In [ ]:
## Note: Please double chech the paths to the respective dataset.
print("Note: Please double chech the paths to the respective dataset !!")
drive.mount('/content/drive')
filepath = '/content/drive/MyDrive/Colab_Notebooks/Thesis_2022/'
if problem == 'MMB':
  path = '/content/drive/MyDrive/Colab_Notebooks/Thesis_2022/Dataset/topopt_1000_resized/'
  BUFFER_SIZE = 1472 # Size of the traning dataset
elif problem == 'Cantilever':
  path = '/content/drive/MyDrive/Colab_Notebooks/Thesis_2022/Dataset/Canti_resized/'
  BUFFER_SIZE = 836 # Size of the traning dataset
else:
  path = input("Enter path to traning and validation data: ")
print('Path: ', path)
train_path = path  + 'train/'
test_path = path + 'val/'
save_model = '/content/drive/MyDrive/Colab_Notebooks/Thesis_2022/TopOptPix2Pix/Models/'

%cd '/content/drive/MyDrive/Colab_Notebooks/Thesis_2022/TopOptPix2Pix'

In [ ]:
EPOCHS = int(input("Enter the number of Epochs: "))
BATCH_SIZE = int(input("Enter the Batch Size: ")) # Batch size of 1 was recomended in the Pix2Pix paper.

IMG_WIDTH = 256 # Width of the traning image    
IMG_HEIGHT = 256 # Height of the traning image

In [ ]:
def plot_losses(g_loss, d_loss, epoch, dataset, base_path='results/'):
    if not os.path.exists(base_path):
      os.makedirs(base_path)
    """Plot the losses each epoc during training."""
    total_loss = [g[0].numpy() + d[0].numpy() for g, d in zip(g_loss, d_loss)]
    g_loss = [g[0].numpy() for g in g_loss]
    d_loss = [d[0].numpy() for d in d_loss]
    plt.figure(figsize=(10,5))
    plt.plot(g_loss, label='gen')
    plt.plot(d_loss, label='disc')
    plt.plot(total_loss, label='total')
    plt.legend()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('training loss')
    plt.savefig(base_path + '/losses.png')
    plt.close()

    def smooth(y, box_pts):
        box = np.ones(box_pts)/box_pts
        y_smooth = np.convolve(y, box, mode='same')
        return y_smooth
    xs = [_ for _ in range(len(d_loss))]
    if len(xs) > 20:
        window=5
        f, axarr = plt.subplots(2, figsize=(8,4), sharex=True)
        axarr[0].plot(d_loss, alpha=0.4, linewidth=5)
        axarr[0].plot(xs[window:-window], smooth(d_loss, window)[window:-window], c='C0')
        axarr[0].set_title('Discriminator')
        axarr[0].set_yscale('log')
        axarr[0].set_ylabel('loss')
        axarr[1].plot(g_loss, alpha=0.4, linewidth=5)
        axarr[1].plot(xs[window:-window], smooth(g_loss, window)[window:-window], c='C1')
        axarr[1].set_title('Generator')
        axarr[1].set_xlabel('Epoch')
        axarr[1].set_ylabel('loss')
        plt.savefig(base_path + '/loss_tracking.png')
        plt.close()

In [ ]:
def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  w = tf.shape(image)[1]

  w = w // 2
  real_image = image[:, :w, :]
  input_image = image[:, w:, :]

  input_image = tf.cast(input_image, tf.float32)
  real_image = tf.cast(real_image, tf.float32)

  return input_image, real_image


In [ ]:
if problem == 'MMB':
  re, inp = load(path + 'train/6-0,497.jpg')
else:  
  re, inp = load(path + 'train/151-0,244.jpg')
# casting to int for matplotlib to show the image
plt.figure()
plt.imshow(inp/255.0)
plt.show()
plt.figure()
plt.imshow(re/255.0)
plt.show()

In [ ]:
def resize(input_image, real_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return input_image, real_image

def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]

# normalizing the images to [-1, 1]

def normalize(input_image, real_image):
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1

  return input_image, real_image

@tf.function()
def random_jitter(input_image, real_image):
  # resizing to 286 x 286 x 3
  a = input_image
  b = real_image
  input_image, real_image = resize(input_image, real_image, 286, 286)

  # randomly cropping to 256 x 256 x 3
  input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) > 0.5:
    # random mirroring
    input_image = tf.image.flip_left_right(input_image)
    real_image = tf.image.flip_left_right(real_image)

  return a, b

plt.figure(figsize=(6, 6))
for i in range(4):
  rj_inp, rj_re = random_jitter(inp, re)
  plt.subplot(2, 2, i+1)
  plt.imshow(rj_re/255.0)
  plt.axis('off')
plt.show()

def load_image_train(image_file):
  real_image, input_image= load(image_file)
  real_image, input_image = random_jitter(real_image, input_image)
  real_image, input_image = normalize(real_image, input_image)

  return input_image, real_image

def load_image_test(image_file):
  real_image, input_image = load(image_file)
  real_image, input_image = resize(real_image, input_image,
                                   IMG_HEIGHT, IMG_WIDTH)
  real_image, input_image = normalize(real_image, input_image)

  return input_image, real_image


In [ ]:
"""## Input Pipeline"""

train_dataset = tf.data.Dataset.list_files(path + 'train/*.jpg')
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.map(load_image_train,
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.list_files(path + 'val/*.jpg')
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(BATCH_SIZE)


In [ ]:
for a, b in train_dataset.take(1): #Take subsets the dataset into at most __1__ element (Seems to randomly create it)
  plt.figure()
  plt.imshow(tf.squeeze(a, [0]))
  plt.axis('off')
  plt.show()
  plt.figure()
  plt.imshow(tf.squeeze(b, [0]))
  plt.axis('off')
  plt.show()


In [ ]:
for a, b in test_dataset.take(1): #Take subsets the dataset into at most __1__ element (Seems to randomly create it)
  plt.figure()
  plt.imshow(tf.squeeze(a, [0]))
  plt.axis('off')
  plt.show()
  plt.figure()
  plt.imshow(tf.squeeze(b, [0]))
  plt.axis('off')
  plt.show()

In [ ]:
#RGB
OUTPUT_CHANNELS = 3
# Weights initializer for the layers.
kernel_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
# Gamma initializer for instance normalization.
gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
# Epsilon for BatchNorm
eps = 1e-05
# Momentum for BatchNorm
mom = 0.1


In [ ]:
class ReflectionPadding2D(layers.Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [
            [0, 0],
            [padding_height, padding_height],
            [padding_width, padding_width],
            [0, 0],
        ]
        return tf.pad(input_tensor, padding_tensor, mode="REFLECT")

In [ ]:
def residual_block(x, activation, kernel_initializer=kernel_init, kernel_size=(3, 3), strides=(1, 1), padding="valid", gamma_initializer=gamma_init, use_bias=False):
  
    dim = x.shape[-1]
    input_tensor = x

    x = ReflectionPadding2D()(input_tensor)
    x = layers.Conv2D(dim, kernel_size, strides=strides, kernel_initializer=kernel_initializer, padding=padding, use_bias=use_bias)(x)
    x = layers.BatchNormalization(momentum=mom, epsilon=eps)(x)
    x = activation(x)

    x = ReflectionPadding2D()(x)
    x = layers.Conv2D(dim, kernel_size, strides=strides, kernel_initializer=kernel_initializer, padding=padding, use_bias=use_bias)(x)
    x = layers.BatchNormalization(momentum=mom, epsilon=eps)(x)
    x = layers.add([input_tensor, x])
    return x

In [ ]:
def downsample(x, filters, activation, kernel_initializer=kernel_init, kernel_size=(3, 3), strides=(2, 2), padding="same", gamma_initializer=gamma_init, use_bias=False):
    x = layers.Conv2D(filters, kernel_size, strides=strides, kernel_initializer=kernel_initializer, padding=padding, use_bias=use_bias)(x)
    x = layers.BatchNormalization(momentum=mom, epsilon=eps)(x)
    if activation:
        x = activation(x)
    return x

In [ ]:
def upsample(x, filters, activation, kernel_size=(3, 3), strides=(2, 2), padding="same", kernel_initializer=kernel_init, gamma_initializer=gamma_init, use_bias=False):
    x = layers.Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding, kernel_initializer=kernel_initializer, use_bias=use_bias)(x)
    x = layers.BatchNormalization(momentum=mom, epsilon=eps)(x)
    if activation:
        x = activation(x)
    return x

In [ ]:
def resnet_generator(
    filters=64,
    num_downsampling_blocks=2,
    num_residual_blocks=50,
    num_upsample_blocks=2,
    gamma_initializer=gamma_init,
    name='resnet_generator',
):
    img_input = layers.Input(shape=[256, 256, 3], name="img_input")
    x = ReflectionPadding2D(padding=(3, 3))(img_input)
    x = layers.Conv2D(filters, 7, kernel_initializer=kernel_init, use_bias=False)(x)
    x = layers.BatchNormalization(momentum=mom, epsilon=eps)(x)
    x = layers.Activation("relu")(x)

    # Downsampling
    for _ in range(num_downsampling_blocks):
        filters *= 2
        x = downsample(x, filters=filters, activation=layers.Activation("relu"))

    # Residual blocks
    for _ in range(num_residual_blocks):
        x = residual_block(x, activation=layers.Activation("relu"))

    # Upsampling
    for _ in range(num_upsample_blocks):
        filters //= 2
        x = upsample(x, filters, activation=layers.Activation("relu"))

    # Final block
    x = ReflectionPadding2D(padding=(3, 3))(x)
    x = layers.Conv2D(3, 7, padding="valid")(x)
    x = layers.Activation("tanh")(x)

    model = keras.models.Model(img_input, x, name=name)
    return model
print(inp[tf.newaxis,...].shape)
generator = resnet_generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)
generator.summary()
# visualkeras.layered_view(generator, legend=True, draw_volume=False)

In [ ]:
gen_output = generator(inp[tf.newaxis, ...], training=False)
plt.imshow(gen_output[0, ...])

In [ ]:
def discriminator(filters=64, kernel_initializer=kernel_init, num_downsampling=3, name='discriminator'):    
    inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
    tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')
    concat_input = tf.keras.layers.concatenate([inp, tar])  # (batch_size, 256, 256, channels*2)
    
    x = layers.Conv2D(filters, 4, strides=(2, 2), padding="same", kernel_initializer=kernel_initializer)(concat_input)
    x = layers.LeakyReLU(0.2)(x)

    num_filters = filters
    for num_downsample_block in range(3):
        num_filters *= 2
        if num_downsample_block < 2:
            x = downsample(x, filters=num_filters, activation=layers.LeakyReLU(0.2), kernel_size=(4, 4), strides=(2, 2))
        else:
            x = downsample(x, filters=num_filters, activation=layers.LeakyReLU(0.2), kernel_size=(4, 4), strides=(1, 1))

    x = layers.Conv2D(1, 4, strides=(1, 1), padding="same", kernel_initializer=kernel_initializer)(x)

    model = keras.models.Model(inputs=[inp, tar], outputs=x, name=name)
    return model

In [ ]:
discriminator = discriminator()
# tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
dis_input = inp[tf.newaxis, ...]
disc_out = discriminator([dis_input, gen_output], training=False)
plt.imshow(disc_out[0, ..., -1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()

In [ ]:
# Loss Functions

LAMBDA = 100

loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# accuracy_object = tf.keras.metrics.categorical_accuracy()

def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss
  return total_disc_loss

def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss

generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)



In [ ]:
## Checkpoints

checkpoint_dir = './training_checkpoints_{}_{}_{}'.format(problem, EPOCHS, BATCH_SIZE)
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  checkpoint.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')


In [ ]:
def generate_images(model, test_input, tar, base_path='', epoch=0):

  if not os.path.exists(base_path):
    os.makedirs(base_path)

  prediction = model(test_input, training=True)
  plt.figure(figsize=(15,8))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.savefig(base_path + '/result_{}.png'.format(epoch))
  plt.close()


In [ ]:

@tf.function
def train_step(input_image, target, g_loss_metric, d_loss_metric):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)


  g_loss_metric.update_state(gen_loss)
  d_loss_metric.update_state(disc_loss)

  generator_gradients = gen_tape.gradient(gen_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))


In [ ]:
def train(dataset, epochs):
  g_loss_log = []
  d_loss_log = []
  g_loss_metric = tf.keras.metrics.Mean(name='g_train_loss')
  d_loss_metric = tf.keras.metrics.Mean(name='d_train_loss')
  accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
  for epoch in range(epochs):
    start = time.time()
    # Reset the metrics
    g_loss_metric.reset_states()
    d_loss_metric.reset_states()
    counter = 0
    for input_image, target in dataset:
      # print("Image Size : ", tf.shape(input_image))
      # print("Image Size : ", tf.shape(target))
      train_step(input_image, target, g_loss_metric, d_loss_metric)
      counter += 1
      if counter > 100:
        break
    # Get the metric results
    g_mean_loss = g_loss_metric.result()
    d_mean_loss = d_loss_metric.result()
    g_loss_log.append([g_mean_loss])
    d_loss_log.append([d_mean_loss])

    print('Epoch: ', epoch)
    print('  loss (g) (d) (g+d):     {:.3f}, {:.3f}, {:.3f}'.format(g_mean_loss, d_mean_loss, g_mean_loss + d_mean_loss))

    if (epoch) % 10 == 0:
      for i, (inp, tar) in enumerate(test_dataset.take(5)):
        generate_images(generator, inp, tar, base_path='results_{}_{}_{}/{}'.format(problem, EPOCHS, BATCH_SIZE, i), epoch=epoch)
        plot_losses(g_loss_log, d_loss_log, epoch=epoch, dataset='dataset', base_path='results_{}_{}_{}'.format(problem, EPOCHS, BATCH_SIZE))

    # Saving (checkpoint) the model every 20 epochs
    if (epoch + 1) % 20 == 0:
    
      ckpt_save_path = ckpt_manager.save()
      print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))


In [ ]:
start_time = time.time()
train(train_dataset, EPOCHS)
end_time = time.time()
print("Average Time Taken Per Epoch: ", (end_time - start_time)/EPOCHS)
generator.save(filepath=save_model + '/ResNet50_Generator', save_format="tf")

Epoch:  0
  loss (g) (d) (g+d):     53.806, 0.994, 54.801
Time taken for epoch 1 is 57.14815306663513 sec

Epoch:  1
  loss (g) (d) (g+d):     55.091, 0.513, 55.603
Time taken for epoch 2 is 42.99859023094177 sec

Epoch:  2
  loss (g) (d) (g+d):     53.320, 0.656, 53.977
Time taken for epoch 3 is 43.36454892158508 sec

Epoch:  3
  loss (g) (d) (g+d):     49.722, 0.439, 50.161
Time taken for epoch 4 is 43.79013800621033 sec

Epoch:  4
  loss (g) (d) (g+d):     54.635, 0.360, 54.996
Time taken for epoch 5 is 44.06349277496338 sec

Epoch:  5
  loss (g) (d) (g+d):     50.676, 0.177, 50.852
Time taken for epoch 6 is 44.328449726104736 sec

Epoch:  6
  loss (g) (d) (g+d):     43.232, 0.763, 43.995
Time taken for epoch 7 is 44.49200987815857 sec

Epoch:  7
  loss (g) (d) (g+d):     43.140, 0.485, 43.625
Time taken for epoch 8 is 44.66976976394653 sec

Epoch:  8
  loss (g) (d) (g+d):     35.069, 0.692, 35.761
Time taken for epoch 9 is 44.613871812820435 sec

Epoch:  9
  loss (g) (d) (g+d):    

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))


In [ ]:

def test_gen(model, test_inp, test_tar, savepath = 'results/final'):
  if not os.path.exists(savepath):
    os.makedirs(savepath)
  prediction = model(test_inp, training=True)
  plt.figure(figsize=(15,8))

  display_list = [test_inp[0], test_tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']
  for i in range(3):

    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.savefig(savepath + '/result_test.png')
  plt.close()


In [ ]:
# Testing
for inp, tar in test_dataset.take(1):
  test_gen(generator, inp, tar)